# Training and Deploying A Deep Learning Model in Keras MobileNet V2 and Heroku: A Step-by-Step Tutorial 

Today, we going to learn how to train a deep learning model and deploy it. It's important to understand the process of training a model in order to apply to your specific domain. As well, After the model is trained and ready to be used, the model need to be easily called via API. Therefore, I am going to walk you through the entire process and show you how after training a model we can with ease deploy to Heroku.

I am going to divide the tutorial into 2 parts:

**Part 1:**
- Prepare data for training 
- Trained a deep learning model

**Part 2:**
- Serve the model with [Tensorflow Serving](https://www.tensorflow.org/serving/).
- Deploy to [Heroku](https://www.heroku.com/).

**In order to fully benefits from this blog:**
- You should be familiar with python.
- You should have some understanding of what deep learning and neural network are.

**Here are the list of what we are going to use:**
- [Keras 2.2](https://keras.io/)is a high-level neural networks API, written in Python and capable of running on top of TensorFlow.
- [Tensorflow 1.11](https://www.tensorflow.org/) is an open-source machine learning library for research and production. Tensorflow is Google's attempt to put the power of Deep Learning into the hands of developers around the world.[2] As well, TensorFlow offers APIs for beginners and experts to develop for desktop, mobile, web, and cloud.
- [Python 3.6](https://www.python.org/downloads/release/python-367/)
- [scikit-image 0.14](https://scikit-image.org/)  is a collection of algorithms for image processing
- [scikit-learn 0.19](http://scikit-learn.org/) Machine Learning in Python
- [pillow 4.1](https://pillow.readthedocs.io/en/4.1.x/)  is the Python Imaging Library


Before we start let's answer some question that may come to your mind

### Why Tensorflow?
There are many reasons why Tensorflow should be selected as deep learning framework. I found that this github project ([easy-tensorflow](https://github.com/easy-tensorflow/easy-tensorflow#why-use-tensorflow)) answers the question 'why' very well and cover all the points I wanted to mention. 

### What is heroku? why heroku? 
Heroku is a platform as a service (PaaS) that enables developers to build, run, and operate applications entirely in the cloud. I am going to use Heroku because it's free to use for development and testing, easy to deploy (does not require much work), support many languages by default, and lastly I could not found much resources about deploying Tensorflow model to Heroku. Here is a nice article [An Introduction to Heroku](https://medium.com/@GoRadialspark/an-introduction-to-heroku-c11c6fcbffa)


## Prepare data for training

### why do we need to prepare data for training?

TODO 

For the sake of simplicity, I am going to use the Fashion-MNIST dataset because it is already optimized and labeled for a classification problem. 
Read more about the Fashion-MNIST dataset in this paper [here](https://arxiv.org/abs/1708.07747).
To download dataset and see other examples [here](https://github.com/zalandoresearch/fashion-mnist)

The Fashion-MNIST dataset has 70,000 grayscale, 
(28x28px) images separated into the following categories:

|Label|Description|  
|-  |      -      |
| 0 | T-shirt/top | 
| 1 | Trouser     | 
| 2 | Pullover    |  
| 3 | Dress       | 
| 4 | Coat        | 
| 5 | Sandal      | 
| 6 | Shirt       |
| 7 | Sneaker     | 
| 8 | Bag         | 
| 9 | Ankle boot  | 


Now let's start downloading the dataset.
Fortunately, many deep learning (DL) frameworks support Fashion-MNIST dataset out of the box, including Keras in our case.

In [ ]:
from keras.datasets.fashion_mnist import load_data

# Load the fashion-mnist train data and test data
(x_train, y_train), (x_test, y_test) = load_data()

print("x_train shape:", x_train.shape, "y_train shape:", y_train.shape)

By default `load_data()` returns training and testing dataset. It is essential to split the dataset into training and testing.    
**Train data**: This data to train the Neural Network (NN).   
**Test**: To valid the Neural Network during the training phase, by tuning and re-adjust the hyperparameters. Hyperparameter is a parameter whose value is set before the learning process begins.

**However sometime you see Train, Valid and Test dataset, why?**

Glad you ask!

After training a Neural Network, we run the trained model against our validation dataset to make sure that the model is generalized and is not overfitting.

**What is overfitting? :)**

Overfitting basically means a model predict the right result when it tests against the train data, but it fails otherwise. However, if a model predicts the incorrectly result for the train data, then it is called underfitting. Here is a nice explanation of [overfitting and underfitting](https://medium.com/greyatom/what-is-underfitting-and-overfitting-in-machine-learning-and-how-to-deal-with-it-6803a989c76).


Thus, we use the validation to detect overfitting or underfitting. But, most of the time we train the model multiple times in order to have higher score in the train and valid datasets. Although, this process of training multiple is required in many use cases, we need to be carefully that we don't end up overfitting in validation set. To make sure the both sets were trained properly, we use the third dataset (Test).

## Helper function to display imags

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

def show_images(images):
    """
    images : numpy arrays
    """
    n_images = len(images)
    titles = ['(%d)' % i for i in range(1, n_images + 1)]
    num = 5
    iter_num = np.ceil(n_images / num).astype(int)
    for i in range(iter_num):
        fig = plt.figure()
        sub_images = images[i * num:num * (i + 1)]
        sub_titles = titles[i * num:num * (i + 1)]
        for n, (image, title) in enumerate(zip(sub_images, sub_titles)):
            a = fig.add_subplot(1, np.ceil(len(sub_images)), n + 1)
            if image.ndim == 2:
                plt.gray()
            a.set_title(title, fontsize=15)
            plt.imshow(image)


### Show samples of data

In [ ]:
show_images(x_train[:10])

### Data normalization
Normalize the data dimensions so that they are of approximately the same scale.In general, normalization makes very deep NN easier to train, special in Convolutional and Recurrent neural network.

Here is a nice explanation [video](https://www.coursera.org/lecture/deep-neural-network/normalizing-activations-in-a-network-4ptp2) and an [article](https://medium.com/@darrenyaoyao.huang/why-we-need-normalization-in-deep-learning-from-batch-normalization-to-group-normalization-d06ea0e59c17)

In [ ]:
norm_x_train = x_train.astype('float32') / 255
norm_x_test = x_test.astype('float32') / 255
# dsiplay images
show_images(norm_x_train[:10])

### Convert labels (y_train and y_test) to one hot encoding
A one hot encoding is a representation of categorical variables as binary vectors.  
[Here is the full explanation](https://machinelearningmastery.com/how-to-one-hot-encode-sequence-data-in-python/) If you would like to have a deep understanding.

In [ ]:
from keras.utils import to_categorical
encoded_y_train = to_categorical(y_train, num_classes=10, dtype='float32')
encoded_y_test = to_categorical(y_test, num_classes=10, dtype='float32')

### Resize images & convert to 3 channel (RGB)

[MobileNet V2](https://keras.io/applications/#mobilenetv2) model accepts one of the following formats: (96, 96), (128, 128), (160, 160),(192, 192), or (224, 224). In addition, the image has to be 3 channel (RGB) format. Therefore, We need to resize & convert our images. from (28 X 28) to (96 X 96 X 3).

In [ ]:
from skimage.transform import resize

target_size = 96

def preprocess_image(x):
    # Resize the image to have the shape of (96,96)
    x = resize(x, (target_size, target_size),
            mode='constant',
            anti_aliasing=False)
    
    # convert to 3 channel (RGB)
    x = np.stack((x,)*3, axis=-1) 
    
    # Make sure it is a float32, here is why 
    # https://www.quora.com/When-should-I-use-tf-float32-vs-tf-float64-in-TensorFlow
    return x.astype(np.float32)

Running the previous code in all our data data, it may eat up a lot of memory resources; therefore, we are going to use generator.   
[Python Generator](https://www.programiz.com/python-programming/generator) is a function that returns an object (iterator) which we can iterate over (one value at a time).

In [ ]:
from sklearn.utils import shuffle
def load_data_generator(x, y, batch_size=64):
    num_samples = x.shape[0]
    while 1:  # Loop forever so the generator never terminates
        try:
            shuffle(x)
            for i in range(0, num_samples, batch_size):
                x_data = [preprocess_image(im) for im in x[i:i+batch_size]]
                y_data = y[i:i + batch_size]
            
                # convert to numpy array since this what keras required
                yield shuffle(np.array(x_data), np.array(y_data))
        except Exception as err:
            print(err)

## Train a Deep Learning model

There are many technique of train the model, today I am going to convert one of them, and I believe that it is one of the most important method or strategy, it is called Transfer Learning.

### Transfer Learning
Transfer learning in deep learning means to transfer a knowledge from one domain to a similar one. In our example, I have chosen MobileNet V2 model because it's faster to train and small in size. And most important, MobileNet is pre-trained with [ImageNet dataset](http://www.image-net.org/).

> ImageNet is an image dataset organized according to the WordNet hierarchy. Each meaningful concept in WordNet, possibly described by multiple words or word phrases, is called a "synonym set" or "synset". There are more than 100,000 synsets in WordNet, majority of them are nouns (80,000+). In ImageNet, we aim to provide on average 1000 images to illustrate each synset. Images of each concept are quality-controlled and human-annotated.

Since our dataset is kind of subset of ImageNet dataset, then we are going to transfer the knowledge of this model onto our datasets. And if you feel like to dive into it more, which I encourage you, here is a nice article that explain it in more details [A Gentle Introduction to Transfer Learning for Deep Learning](https://machinelearningmastery.com/transfer-learning-for-deep-learning/)


In [ ]:
from keras.applications.mobilenetv2 import MobileNetV2
from keras.layers import Dense, Input, Dropout
from keras.models import Model

def build_model( ):
    input_tensor = Input(shape=(target_size, target_size, 3))
    base_model = MobileNetV2(
        include_top=False,
        weights='imagenet',
        input_tensor=input_tensor,
        input_shape=(target_size, target_size, 3),
        pooling='avg')

    for layer in base_model.layers:
        layer.trainable = True  # trainable has to be false in order to freeze the layers
        
    op = Dense(256, activation='relu')(base_model.output)
    op = Dropout(.25)(op)
    
    ##
    # softmax: calculates a probability for every possible class.
    #
    # activation='softmax': return the highest probability;
    # for example, if 'Coat' is the highest probability then the result would be 
    # something like [0,0,0,0,1,0,0,0,0,0] with 1 in index 5 indicate 'Coat' in our case.
    ##
    output_tensor = Dense(10, activation='softmax')(op)

    model = Model(inputs=input_tensor, outputs=output_tensor)


    return model

The below code is important to understand when working using Transfer Learning as a technique.
```
for layer in base_model.layers:
  # trainable has to be false in order to freeze the layers
  layer.trainable = False # or True
```

As using a  pre-trained model (e.g. MobileNetV2 in our case), you need to pay close a attention  to a concept call Fine Tuning.

> **'Fine Tuning'**, generally, is when we freeze the weights of all the layers of the pre-trained neural networks (on dataset A [e.g. ImageNet]) except the penultimate layer and train the neural network on dataset B [e.g. Fashion-MNIST], just to learn the representations on the penultimate layer. We usually replace the last (softmax) layer with another one of our choice (depending on the number of outputs we need for the new problem.[3]

In our case, we have 10 classes, so we have the following  
`output_tensor = Dense(10, activation='softmax')(op)`

**When do we use need Fine Tuning?**
- When you have small datasets (e.g. few 1000s)
- When the dataset used to train the pre-trained model is very  similar or the same as the new dataset. 

An obvious use case to use Fine Tuning is when building a model predicts only sub classes (20) out of 1000 of ImageNet dataset as dataset B (sub of ImageNet) is a subset of dataset A (ImageNet).

In [ ]:
from keras.optimizers import Adam
model = build_model()
model.compile(optimizer=Adam(),
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])

Now we build and compile the model. Some definition use when compile a model:
> The Adam optimization algorithm is an extension to stochastic gradient descent that has recently seen broader adoption for deep learning applications in computer vision and natural language processing.[4]

> A loss function (categorical_crossentropy) is a measure of how good a prediction model does in terms of being able to predict the expected outcome. [5]

> categorical_accuracy is a metric function that is used to judge the performance of your model.[6]

In [ ]:
train_generator = load_data_generator(norm_x_train, encoded_y_train, batch_size=64)

model.fit_generator(
    generator=train_generator,
    steps_per_epoch=5,
    verbose=1,
    epochs=2)

It is an essential to the understand the following when training any deep leaning model.
> Epoch is when an entire dataset is passed forward and backward through the neural network only once. [7]

> Batch Size is the total number of training examples present in a single batch [7]. And it goes a long with python generate mention previously 

> Iterations (steps_per_epoch)  is the number of batches needed to complete one epoch [7]. 
And to have more details understand, I suggest read the following article Epoch vs Batch Size vs Iterations.

### Test
94% I got after 5 epoch of training, now I would like to see how I did in the test dataset

In [ ]:
test_generator = load_data_generator(norm_x_test, encoded_y_test, batch_size=64)
model.evaluate_generator(generator=test_generator,
                         steps=900,
                         verbose=1)

It was 86%, seems reasonable for the amount I spent to train, 1 hour in CPU machine.

**Things to you could try to improve the accuracy**

- Choose more epoch, 10 as an example
- Try freeze the layers layer.trainable = False. you could also get less.
- Choose bigger batch_size, 128 as an example
- Select different optimizer, Nadam, for instance. However changing the optimizer sometime does not give mush improving.

## Save the model
Make sure you save the model, because we are going to use in next part

In [ ]:
model_name = "tf_serving_keras_mobilenetv2"
model.save(f"models/{model_name}.h5")

## Part 2

## Makes Model ready to tensorflow serving

### Tensorflow serving 
[TensorFlow Serving](https://www.tensorflow.org/serving/overview) is a flexible, high-performance serving system for machine learning models, designed for **production** environments.


#### Load the saved model from Part 1


In [ ]:
from keras.models import load_model
model = load_model(f"models/{model_name}.h5")

### Build & Save model to be tensorflow serving ready


In [ ]:
import os
import tensorflow as tf
# Import the libraries needed for saving models
# Note that in some other tutorials these are framed as coming from tensorflow_serving_api which is no longer correct
from tensorflow.python.saved_model import builder as saved_model_builder
from tensorflow.python.saved_model import tag_constants, signature_constants, signature_def_utils_impl

# images will be the input key name
# scores will be the out key name
prediction_signature = tf.saved_model.signature_def_utils.predict_signature_def(
    {
    "images": model.input
    }, {
    "scores": model.output
    })

# export_path is a directory in which the model will be created
export_path = os.path.join(
    tf.compat.as_bytes('models/export/{}'.format(model_name)),
    tf.compat.as_bytes('1'))

builder = saved_model_builder.SavedModelBuilder(export_path)

sess = keras.backend.get_session()

# Add the meta_graph and the variables to the builder
builder.add_meta_graph_and_variables(
    sess, [tag_constants.SERVING],
    signature_def_map={
        'prediction': prediction_signature,
    })
# save the graph
builder.save()

## Setting up Heroku & Docker
Nice work, almost there, now we are going to deploy our model to heroku.

[Heroku](https://www.heroku.com/) is a cloud platform as a service supporting several programming languages.
[docker](https://www.docker.com/) allows us to package all the necessary libraries and programer into a container. Here is a nice [Introduction to Containers, VMs and Docker](https://medium.freecodecamp.org/a-beginner-friendly-introduction-to-containers-vms-and-docker-79a9e3e119b)


- [Install docker for macOS and windows](https://www.docker.com/products/docker-desktop)
- A little more work for Ubuntu users but still straightforward [Install docker for Ubuntu](https://docs.docker.com/install/linux/docker-ce/ubuntu/#upgrade-docker-ce)
- [Signup to Heroku](https://signup.heroku.com/)
- [Install heroku-cli](https://devcenter.heroku.com/articles/heroku-cli#download-and-install)

### After you have installed docker and heroku-cli.

Run the following to make sure docker & heroku have been installed correctly
```
> docker ps 
CONTAINER ID IMAGE COMMAND CREATED STATUS PORTS NAMES

> heroku --version
heroku/7.18.3 darwin-x64 node-v10.12.0

# make sure you have logged in to your heroku account
heroku login
# Output should have:   
Logged in as xxxxx@xxx.xx
```


### Deploy the model to Heroku

#### Download tensorflow serving image from `hub.docker.com`
Because tensorflow serving docker image was not optimized for heroku. 

I have created a dockerfile that following heroku instructure.
cleck [here](https://github.com/malnakli/ML/tf_servning_keras_mobilenetv2/Dockerfile) to look at.
Also I have pushed the a docker image that ready to deploy to heroku, which already have the trained model. 
`docker pull malnakli/ml:tf-serving-heroku-1.11`

However, I will walk you through how to do build the image that has your trained model

Run the following:
And make sure you are in the right folder before running any command `cd ML/tf_servning_keras_mobilenetv2`

Build docker image:
`docker build -t tf-serving-heroku-1.11 .`

Once the image build. you can run it locally if you would like, otherwise go to deploy section.
`docker run -p 8501:8501 -e PORT=8501 -t tf-serving-heroku-1.11`

If you see the following the last output, then it works.
```
2018-10-27 21:17:47.515120: I tensorflow_serving/model_servers/server.cc:301] Exporting HTTP/REST API at:localhost:8501 ...
```
#### Deploy

##### Log in to Container Registry:
`heroku container:login`

##### Create a heroku app
`heroku create ${YOUR_APP_NAME}`

#### Push the docker image to heroku
`heroku  container:push web -a ${YOUR_APP_NAME}`   
`heroku container:release web -a ${YOUR_APP_NAME}`

### Call the model
Before calling the model,let's understand how Tensorflow Serving RESTful API works:
```
POST http://host:port/<URI>:<VERB>

URI: /v1/models/${MODEL_NAME}[/versions/${MODEL_VERSION}]
VERB: classify|regress|predict
```
In our case the request will look like as:

> `http://localhost:8501//v1/models/tf_servning_keras_mobilenetv2/versions/1:predict`


Also, the JSON data that is sent to TensorFlow Model Server has to be structured in a very particular way
```
{
  // (Optional) Serving signature to use.
  // If unspecified default serving signature is used.
  "signature_name": <string>,

  // Input Tensors in row ("instances") or columnar ("inputs") format.
  // A request can have either of them but NOT both.
  "instances": <value>|<(nested)list>|<list-of-objects>
  "inputs": <value>|<(nested)list>|<object>
}
```

In order case:
```
{
  "signature_name":'prediction',
  "instances": [{"images":image.tolist()}]
}
```

And you can see the full documentation about RESTful API in [Here](https://www.tensorflow.org/serving/api_rest). 

Now I will show you how we can call the model.    
However, before calling our model, we need to resize the image and convert them to float32 
by using image_preprcess(image) function from part 1

In [ ]:
from urllib import request
from PIL import Image

image_url = "https://cdn.shopify.com/s/files/1/2029/4253/products/Damb_Back_2a3cc4cc-06c2-488e-8918-2e7a1cde3dfc_530x@2x.jpg"
image_path = f"tmp/{image_url.split('/')[-1]}"
# download image
with request.urlopen(url=image_url, timeout=10) as response:
    data = response.read()
    with open(image_path, 'wb') as f:
        f.write(data)

# convert image to grayscale.
image = Image.open(image_path).convert('L')
# resize the image to 28 28 to make sure it is similar to our dataset
image.thumbnail((28,28))
image = preprocess_image(np.array(image))
print(image.shape)
show_images([image])

If you would like how we can call our model in `curl` here you go:
```
curl -X POST \
  https:// ${YOUR_APP_NAME}.herokuapp.com/v1/models/tf_servning_keras_mobilenetv2/versions/1:predict  \
  -d '{"signature_name":"prediction","instances":[{"images":image.tolist()}]}'
```

In [ ]:
import requests
import json
"""
NOTE:
change https://tf-servning-keras-mobilenetv2.herokuapp.com to your url or 
if you ran the docker locally, then replace with http://localhost:8501
""" 
url = "https://tf-servning-keras-mobilenetv2.herokuapp.com"
full_url = f"{url}/v1/models/tf_servning_keras_mobilenetv2/versions/1:predict"
data = {"signature_name":"prediction",
        "instances":[{"images":image.tolist()}]}
data = json.dumps(data)

In [ ]:
import sys
labels = ['T-shirt/top' ,'Trouser','Pullover','Dress','Coat','Sandal','Shirt','Sneaker','Bag', 'Ankle boot']
try:
    response = requests.post(full_url,data=data)
    response = response.json()
    highest_index = np.argmax(response['predictions'])
    print(labels[highest_index])
except:
    print(sys.exc_info()[0])

## Conclusion 

In the in I hope you have enjoyed the tutorial, if you will to see more like this please following and clap :). And if you have any question or suggests, please leave them down in the comments

# References:
- [1] [Epoch vs Batch Size vs Iterations
](https://towardsdatascience.com/epoch-vs-iterations-vs-batch-size-4dfb9c7ce9c9)
- [2] [Gentlest Introduction to Tensorflow #1
](https://medium.com/all-of-us-are-belong-to-machines/the-gentlest-introduction-to-tensorflow-248dc871a224)

Here are some articles that it convert similar aspect cover in this article.

- [Fashion-MNIST with tf.Keras](https://medium.com/tensorflow/hello-deep-learning-fashion-mnist-with-keras-50fcff8cd74a)
- [A Comprehensive guide to Fine-tuning Deep Learning Models in Keras  ](https://flyyufelix.github.io/2016/10/03/fine-tuning-in-keras-part1.html)
- [Transfer Learning and Fine Tuning: Let's discuss](https://www.linkedin.com/pulse/transfer-learning-fine-tuning-lets-discuss-arun-das/)
- [Serving Image-Based Deep Learning Models with TensorFlow-Serving’s RESTful API](https://medium.com/@tmlabonte/serving-image-based-deep-learning-models-with-tensorflow-servings-restful-api-d365c16a7dc4)
- [How to Setup Tensorflow Serving For Production](https://medium.com/@brianalois/how-to-setup-tensorflow-serving-for-production-3cc2abf7efa)
- [How to Run Dockerized Apps on Heroku… and it’s pretty sweet] (https://medium.com/travis-on-docker/how-to-run-dockerized-apps-on-heroku-and-its-pretty-great-76e07e610e22)